In [31]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, confusion_matrix, accuracy_score, recall_score
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
import datetime
import os
import json

In [32]:
# Parámetros
sourceDataFolder = "1-source-data"

intermFilesFolder = "2-intermediate-files"
resultsFolder = "3-results"
logsFolder = "4-logs"

dataVisualizationTopLimit = 20

counterIn = 0
counterOut = 0

testSize = 0.2
randomState = 0
partitionsNumber = 0

In [33]:
# Función para leer datos de un archivo csv
def readData(filePath, delimiter, encoding, header):
  data = pd.read_csv(filePath, delimiter=delimiter,encoding=encoding, header=header)
  return data

In [34]:
def preprocessingData(data):
  # Aca no se realizará ni parseo ni nada, ya vendrá desde la fuente
  # Solo se leerá, se dividirá y se entrenará

  # Desordenando la data (filas)
  data = data.sample(frac = 1, random_state=randomState).reset_index(drop=True)

  # Filtrar filas nulas
  #data = data.dropna()

  # Eliminando columnas que no se usan para el modelo
  nonModelColumns = ["postulationDate", "candidateName"]
  # , "lastWorkCenter", "lastWorkPosition"
  data = data.drop(nonModelColumns, axis=1)

  # Aplicando OneHotEncoding a las variables categóricas (transformación a numéricas y normalización)
  categoricalColumns = ["profileName", "residenceCountry", "lastWorkCenter", "lastWorkPosition", "studyCenter", "careerField", "careerStatus", "careerDegree"]
  transformer = make_column_transformer( (OneHotEncoder(sparse=False), categoricalColumns), remainder='passthrough')
  transformed = transformer.fit_transform(data)
  data = pd.DataFrame(transformed, columns=transformer.get_feature_names())

  # Aplicando MinMaxScaler a las variables numéricas (normalización)
  numericNonScaledColumns = ["yearsOfExperience", "worksNumber", "studiesNumber", "technicalSkills", "languages", "anotherSkills", "salary"]
  mms = MinMaxScaler()
  data[numericNonScaledColumns] = mms.fit_transform(data[numericNonScaledColumns])

  # Estandarizando el tipo de la variable objetivo a entero
  objectiveColumn = "hired"
  data[objectiveColumn] = data[objectiveColumn].astype(int)

  # Lectura de las variables de características y objetivo
  X = data.drop([objectiveColumn], axis=1)
  y = data[objectiveColumn]

  return X, y

In [35]:
def splitData(X, y, partitionNumber):
  # Dividiendo los dataframes de entrenamiento y prueba
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testSize, random_state=randomState)

  if partitionNumber != 0:
    # Obteniendo el total de filas del dataframe de testeo
    X_train, X_test, y_train, y_test = X_train.reset_index(drop=True), X_test.reset_index(drop=True), y_train.reset_index(drop=True), y_test.reset_index(drop=True)
    totalRows = len(X_train)

    # Determinando el límite inferior (primera fila) de la partición
    bottomLimit = int(totalRows*testSize*(partitionNumber-1)) + 1

    # Determinando el límite superior (última fila) de la partición
    topLimit = int(totalRows*testSize*partitionNumber)

    # Determinando una lista con todas los numeros de las filas con la partición
    testList = [x for x in range(bottomLimit-1, topLimit)]

    # Determinando los dataframes de las categorías
    X_train, X_test = X_train[~X_train.index.isin(testList)], X_train[X_train.index.isin(testList)]

    # Determinando los dataframes de los objetivos
    y_train, y_test = y_train[~y_train.index.isin(testList)], y_train[y_train.index.isin(testList)]

  return X_train, X_test, y_train, y_test

In [36]:
def createClassifier(modelName, parameters):
  if modelName == "KNN":
    return KNeighborsClassifier(parameters) if parameters is not None else KNeighborsClassifier()
  elif modelName == "LR":
    return LogisticRegression(parameters) if parameters is not None else LogisticRegression()
  elif modelName == "GNB":
    return GaussianNB(parameters) if parameters is not None else GaussianNB()
  elif modelName == "DT":
    return DecisionTreeClassifier(parameters) if parameters is not None else DecisionTreeClassifier()
  elif modelName == "SVM":
    return SVC(parameters) if parameters is not None else SVC()
  elif modelName == "RF":
    return RandomForestClassifier(parameters) if parameters is not None else RandomForestClassifier()
  elif modelName == "GB":
    return GradientBoostingClassifier(parameters) if parameters is not None else GradientBoostingClassifier()
  else:
    return KNeighborsClassifier()

In [37]:
def trainModel(X_train, X_test, y_train, y_test, modelName, parameters = None):
  # Creación del clasificador KNN
  clf = createClassifier(modelName, parameters)

  # Entrenamiento del clasificador KNN
  clf.fit(X_train, y_train)

  # Calculando la predicción del modelo con la data de prueba
  y_pred = clf.predict(X_test)

  return y_test, y_pred

In [38]:
def getMetrics(y_train, y_test, y_pred, startDate, endDate, partitionNumber, algorithm = "KNN"):
  trainRows = len(y_train)
  testRows = len(y_test)

  # Calculando la exactitud del modelo
  accuracy = "{:.2%}".format(accuracy_score(y_test, y_pred))

  # Calculando la precisión del modelo
  precision = "{:.2%}".format(precision_score(y_test, y_pred))

  # Calculando el valor F del modelo
  f1Score = "{:.2%}".format(f1_score(y_test, y_pred))

  # Calculando la sensibilidad del modelo
  recall = "{:.2%}".format(recall_score(y_test, y_pred))

  # Calculando el tiempo de ejecución del modelo
  executionTime = str(int((endDate - startDate).total_seconds() * 1000)) + "ms"

  confussionMatrix = str(confusion_matrix(y_test, y_pred).tolist())
  
  return {
    "algoritmo": algorithm,
    "tipo": "Total de datos" if partitionNumber == 0 else "Particion " + str(partitionNumber),
    "registrosEntrenamiento": trainRows,
    "registrosPrueba": testRows,
    "tiempoEjecucion": executionTime,
    "matrizConfusion": confussionMatrix,
    "exactitud": accuracy,
    "precision": precision,
    "valorF": f1Score,
    "sensibilidad": recall,
  }


In [39]:
def main():
  # Definiendo el inicio del proceso
  startTime = datetime.datetime.now()
  print("Inicio: " + str(startTime))
  print("Se inició el procesamiento")

  # Leyendo la data
  data = readData('1-source-data/result.csv', ',', 'utf-8', 0)

  # Determinando los dataframes de las categorías (X) y el objetivo (y)
  X, y = preprocessingData(data)

  # Creando el arreglo de metricas de cada partición
  metricsList = []

  # Iterando sobre cada partición
  for partitionNumber in range(0,partitionsNumber+1):
    # Separando data para el entrenamiento y testeo
    X_train, X_test, y_train, y_test = splitData(X, y, partitionNumber)

    models = ["KNN", "LR", "GNB", "DT", "SVM", "RF", "GB"]

    for model in models:
      print("Calculando para el algoritmo {}".format(model))

      # Inicio de ejecución del modelo
      startDate = datetime.datetime.now()
      
      # Realizar entrenamiento del modelo
      y_test, y_pred = trainModel(X_train, X_test, y_train, y_test, model)

      # Fin de ejecución del modelo
      endDate = datetime.datetime.now()

      # Obteniendo las métricas de la partición del modelo
      metrics = getMetrics(y_train, y_test, y_pred, startDate, endDate, partitionNumber, model)

      # Añadiendo la métrica de la partición a la lista de métricas
      metricsList.append(metrics)

  # Ordenando las métricas
  metricsList = sorted(metricsList, key=lambda x: (x["algoritmo"], x["tipo"]))

  # Mostrando las métricas
  metricsListPretty = json.dumps(metricsList, indent=4)
  print(metricsListPretty)
    
  # Escribiendo las metricas en un archivo de salida
  with open(os.path.join(resultsFolder, datetime.datetime.now().strftime("%Y-%m-%d %H-%M-%S") + ".json"), "w") as f:
      json.dump(metricsList, f, indent=4, ensure_ascii=False)

  # Definiendo el fin del proceso
  endTime = datetime.datetime.now()
  print("Fin: " + str(endTime))
  print("Tiempo: " + str(endTime-startTime))
  
  # Retornando la lista de métricas
  return metricsList

In [40]:
if __name__ == "__main__":
  main()

Inicio: 2023-05-12 02:55:31.086382
Se inició el procesamiento


c:\Users\Ronaldo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Calculando para el algoritmo KNN
Calculando para el algoritmo LR
Calculando para el algoritmo GNB
Calculando para el algoritmo DT
Calculando para el algoritmo SVM
Calculando para el algoritmo RF
Calculando para el algoritmo GB
[
    {
        "algoritmo": "DT",
        "tipo": "Total de datos",
        "registrosEntrenamiento": 4972,
        "registrosPrueba": 1244,
        "tiempoEjecucion": "17798ms",
        "matrizConfusion": "[[1145, 12], [54, 33]]",
        "exactitud": "94.69%",
        "precision": "73.33%",
        "valorF": "50.00%",
        "sensibilidad": "37.93%"
    },
    {
        "algoritmo": "GB",
        "tipo": "Total de datos",
        "registrosEntrenamiento": 4972,
        "registrosPrueba": 1244,
        "tiempoEjecucion": "26609ms",
        "matrizConfusion": "[[1157, 0], [70, 17]]",
        "exactitud": "94.37%",
        "precision": "100.00%",
        "valorF": "32.69%",
        "sensibilidad": "19.54%"
    },
    {
        "algoritmo": "GNB",
        "tipo":